<p style="font-weight:bold;"> <span style="font-size: 36px"> Ifrs Variable Exporter </span> </p>

# Infrastructure and Configuration

## Initialize Workspace

In [0]:
#!eval-notebook "InitSystemorphToMemoryForTesting"

## Configurations

In [0]:
//var pathToBm = "./Data/ReportBenchmarks/";

In [0]:
int roundingDigits = - (int)Math.Log10(BenchmarkPrecision) + 1;

# Defining Scope for the IfrsVariables

## Defining the Import Args

In [0]:
var exportScope = new ImportArgs[]{
    new ImportArgs ("CH", 2020, 12, (Periodicity)default, null, null),
    new ImportArgs ("CH", 2021, 3, (Periodicity)default, null, null)
    };

# Gathering Data

In [0]:
var args = exportScope[0];
args

In [0]:
await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(new {ReportingNode = args.ReportingNode, Year = args.Year, Month = args.Month, Scenario = args.Scenario});
var ifrsVariables = (await DataSource.Query<IfrsVariable>().ToArrayAsync())
            .Select(v => v with { Value = Math.Round(v.Value, roundingDigits)})
            .OrderBy(v => v.DataNode).ThenBy(v => v.AccidentYear).ThenBy(v => v.EstimateType).ThenBy(v => v.EconomicBasis).ThenBy(v => v.AmountType).ThenBy(v => v.Novelty)
            .ThenBy(v => v.AocType)
            .ToArray();

# Export

## Export SimpleValue

In [0]:
const string fileNamePrefix = "SimpleValue";

In [0]:
await Export.ToCsv(fileNamePrefix + "_" + args.ReportingNode + "_" + args.Year.ToString() + "_" + args.Month.ToString() + "_" + args.Scenario)
                                    .WithTable<ImportArgs>(tableConfig => tableConfig
                                                                    .AtBeginning()
                                                                    .WithName(Main)
                                                                    .WithSource(source => args.RepeatOnce().AsQueryable())
                                                                    .WithColumn(x => x.Periodicity, x => x.Delete())
                                                                    .WithColumn(x => x.ImportFormat, x => x.Delete())
                                                    )
                                    .WithTable<IfrsVariable>(tableConfig => tableConfig
                                                                  .WithSource(source => ifrsVariables.AsQueryable())
                                                                  .WithName(fileNamePrefix)
                                                                  .WithColumn(x => x.Partition, x => x.Delete())
                                                                  .WithColumn(x => x.Id, x => x.Delete())
                                                                 )
                                    .WithSource(DataSource)
                                    .ExecuteAsync()    

## Export by Estimate Type

In [0]:
const string benchmarkFileNamePrefix = "BM";

In [0]:
var estimateType = "BE";

In [0]:
await Export.ToCsv(benchmarkFileNamePrefix + "_" + args.ReportingNode +"_" + args.Year.ToString() + "_" + args.Month.ToString() + "_" + scenario + "_" + estimateType)
                                    .WithTable<ImportArgs>(tableConfig => tableConfig
                                                                    .AtBeginning()
                                                                    .WithName(Main)
                                                                    .WithSource(source => args.RepeatOnce().AsQueryable())
                                                                    .WithColumn(x => x.Periodicity, x => x.Delete())
                                                                    .WithColumn(x => x.ImportFormat, x => x.Delete())
                                                    )
                                    .WithTable<IfrsVariable>(tableConfig => tableConfig
                                    .WithSource(source => ifrsVariables.Where(x => x.EstimateType == estimateType).AsQueryable())
                                                                  .WithName(estimateType)
                                                                  .WithColumn(x => x.Partition, x => x.Delete())
                                                                  .WithColumn(x => x.Id, x => x.Delete())
                                                                 )
                                    .WithSource(DataSource)
                                    .ExecuteAsync()                         